## clinical variable retrieval codebase.
each querey is referencing a sql querey on my computer for ALL patients in the database, then generating a dataframe, then paring that dataframe down to only the patients/icustay_id in our cohort. it's inefficient but works. 

In [11]:
#6-25-18
#the streamlined version of this notebook seeks to accomplish a few tasks:
#organize and standardize all sql code so that i am running a .sql file rather than typing sql code into jupyter
#change all filepaths to match the github linked directory to ensure better version control
###THIS IS STILL A WORK IN PROGRESS

time: 816 µs


In [2]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
#patients of interest from rotation_cohort_generation

final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/16072018_final_pt_df2.csv', index_col=0)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 86.1 ms


## Access MIMIC database and convert it to dataframe in Pandas

In [4]:
import psycopg2
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

time: 3.88 s


In [5]:
#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

def sql_to_df_hadm(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['hadm_id'].isin(hadm_id)]
    return sql_exe_show_df

def clinvar_fxn(var_name, path):
    f= open(path, 'r')
    var = f.read()
    cur.execute('rollback')
    cur.execute(var)
    try:
        df= sql_to_df_icu(sql_exe_show('select * from %s;' %(var_name)))
    except KeyError or NameError:
        try:      
            df= sql_to_df_hadm(sql_exe_show('select * from %s;' %(var_name)))
        except KeyError or NameError:
            df= sql_to_df_patients(sql_exe_show('select * from %s;' %(var_name)))

    print(df.shape)
    return(df)


time: 49.9 ms


## Results in .csv

In [ ]:
#where are these tables? i don't see any code generating this? 

In [95]:
# index = [] 
# index.append(echoTable.columns)
# index.append(heightFirstDayTable.columns)
# index.append(weightFirstDayTable.columns)
# index.append(vitalsFirstDayTable.columns)
# index.append(labsFirstDayTable.columns)
# index.append(bgFirstDayTable.columns)
# index.append(gcsFirstDayTable.columns)
# index.append(rrtFirstDayTable.columns)
# index.append(urineFirstDayTable.columns)
# pd.DataFrame(index).to_csv('index.csv')


NameError: name 'vitalsFirstDay' is not defined

time: 46.7 ms


In [99]:
# # index = [] 
# # index.append(echoDf.columns)
# # index.append(heightFirstDayDf.columns)
# # index.append(weightFirstDayDf.columns)
# # index.append(vitalsFirstDayDf.columns)
# # index.append(labsFirstDayDf.columns)
# # index.append(bgFirstDayDf.columns)
# # index.append(gcsFirstDayDf.columns)
# # index.append(rrtFirstDayDf.columns)
# # index.append(urineFirstDayDf.columns)
# pd.DataFrame(echoDf).to_csv('clinical_csv/echoDf.csv')
# pd.DataFrame(heightFirstDayDf).to_csv('clinical_csv/heightFirstDayDf.csv')
# pd.DataFrame(weightFirstDayDf).to_csv('clinical_csv/weightFirstDayDf.csv')
# pd.DataFrame(vitalsFirstDayDf).to_csv('clinical_csv/vitalsFirstDayDf.csv')
# pd.DataFrame(labsFirstDayDf).to_csv('clinical_csv/labsFirstDayDf.csv')
# pd.DataFrame(bgFirstDayDf).to_csv('clinical_csv/bgFirstDayDf.csv')
# pd.DataFrame(gcsFirstDayDf).to_csv('clinical_csv/gcsFirstDayDf.csv')
# pd.DataFrame(rrtFirstDayDf).to_csv('clinical_csv/rrtFirstDayDf.csv')
# pd.DataFrame(urineFirstDayDf).to_csv('clinical_csv/urineFirstDayDf.csv')


time: 2.75 s


# extracting clinical data for our patients

## Echodata in Noteevents

In [5]:
echodata_df= clinvar_fxn(
    'echodata',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/echodata.sql' 
)

(10137, 18)
time: 34 s


## Weight

In [19]:
# -- This query extracts weights for adult ICU patients on their first ICU day.
# -- It does *not* use any information after the first ICU day, as weight is
# -- sometimes used to monitor fluid balance.

weightfirstday_df= clinvar_fxn(
    'weightfirstday',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/weightfirstday.sql' 
)


(12801, 6)
time: 7.19 s


## Height

In [7]:
# -- This query extracts heights for adult ICU patients.
# -- It uses all information from the patient's first ICU day.
# -- This is done for consistency with other queries - it's not necessarily needed.
# -- Height is unlikely to change throughout a patient's stay.
heightfirstday_df= clinvar_fxn(
    'heightfirstday',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/heightfirstday.sql' 
)

(12801, 4)
time: 2.52 s


# Labs

In [8]:
# -- This query pivots lab values taken in the first 24 hours of a patient's stay
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit

labs_all_nosummary_df= clinvar_fxn(
    'labs_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/labs_all_nosummary.sql' 
)


(3036173, 6)
time: 6min 26s


## Vital Sign

In [9]:
##-- This query pivots the vital signs for the first 24 hours of a patient's stay
##-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

vitals_all_nosummary_df= clinvar_fxn(
    'vitals_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/vitals_all_nosummary.sql' 
)

(14028103, 6)
time: 19min 44s


## Glasgow Coma Scale

In [10]:
gcsall_df= clinvar_fxn(
    'gcsall',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/gcsall.sql' 
)

(12801, 8)
time: 2min 13s


## Renal replacement therapy

In [11]:
rrt_all_df= clinvar_fxn(
    'rrt_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rrt_all.sql' 
)

(12801, 4)
time: 9.48 s


## Urine Output

In [12]:
urine_output_all_df= clinvar_fxn(
    'urine_output_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/urine_output_all.sql' 
)

(73302, 5)
time: 15.8 s


# UTI related variables

In [6]:
uti_all_df= clinvar_fxn(
    'uti_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/uti_all.sql' 
)

(645133, 12)
time: 15min 23s


In [10]:
uti_all_df.head(10)

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code
78,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,431
79,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,5070
80,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,4280
81,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,5849
82,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,2765
83,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,4019
89,9,150750.0,51486,2149-11-10 09:24:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,431
90,9,150750.0,51486,2149-11-10 09:24:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,5070
91,9,150750.0,51486,2149-11-10 09:24:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,4280
92,9,150750.0,51486,2149-11-10 09:24:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,5849


time: 25.2 ms


## Blood Gas Test

In [13]:
bg_all_nosummary_df= clinvar_fxn(
    'bg_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/bg_all_nosummary.sql' 
)

(1705168, 6)
time: 3min 25s


## Vasodil

In [27]:

weightdurations_df= clinvar_fxn(
    'weightdurations',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/weightdurations.sql'
)

epi_dose_df= clinvar_fxn(
    'epinephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/epinephrine_dose.sql'
)
norepi_dose_df= clinvar_fxn(
    'norepinephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/norepinephrine_dose.sql'
)
dopamine_dose_df= clinvar_fxn(
    'dopamine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/dopamine_dose.sql'
)
dobutamine_dose_df= clinvar_fxn(
    'dobutamine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/dobutamine_dose.sql'
)
vasopressin_dose_df= clinvar_fxn(
    'vasopressin_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/vasopressin_dose.sql'
)
phenylephrine_dose_df= clinvar_fxn(
    'phenylephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/phenylephrine_dose.sql'
)

(45448, 4)
(2397, 5)
(44371, 5)
(8900, 5)
(1922, 5)
(3472, 5)
(36608, 5)
time: 52.8 s


In [33]:
ventsettings_df= clinvar_fxn(
    'ventsettings',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/ventsettings.sql'
)
#going from wide format to long:
#pd.melt(ventsettings_df, id_vars=['icustay_id','charttime'])

(688532, 6)
time: 4min 11s


,icustay_id,charttime,variable,value
0,292972.0,2178-03-14 22:00:00,mechvent,0
1,255356.0,2156-04-26 18:00:00,mechvent,0
2,295273.0,2195-09-23 18:26:00,mechvent,1
3,288354.0,2103-11-04 09:40:00,mechvent,0
4,226799.0,2153-08-06 11:03:00,mechvent,1
5,232333.0,2120-08-22 12:00:00,mechvent,0
6,215903.0,2175-06-10 12:59:00,mechvent,1
7,258962.0,2134-08-29 00:33:00,mechvent,1
8,269856.0,2119-02-07 10:00:00,mechvent,0
9,252836.0,2191-08-24 17:23:00,mechvent,1


time: 229 ms


In [39]:
pd.DataFrame(ventsettings_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date))

time: 5.56 s


In [38]:
ventsettings_df.head()
#ventsettings_df.loc[ventsettings_df.loc[:,'icustay_id']==267059.0,'charttime'].describe()

,icustay_id,charttime,mechvent,oxygentherapy,extubated,selfextubated
5,292972.0,2178-03-14 22:00:00,0,1,0,0
6,255356.0,2156-04-26 18:00:00,0,1,0,0
9,295273.0,2195-09-23 18:26:00,1,0,0,0
14,288354.0,2103-11-04 09:40:00,0,1,0,0
20,226799.0,2153-08-06 11:03:00,1,0,0,0


time: 9.14 ms


In [28]:
#adding an identification label column and merging them into 1 df. 
epi_dose_df['label']='epinephrine'
norepi_dose_df['label']='norepinephrine'
dopamine_dose_df['label']='dopamine'
dobutamine_dose_df['label']='dobutamine'
vasopressin_dose_df['label']='vasopressin'
phenylephrine_dose_df['label']='phenylephrine'
vaso_dose_df = pd.concat([epi_dose_df, norepi_dose_df, dopamine_dose_df, dobutamine_dose_df, vasopressin_dose_df,phenylephrine_dose_df ])

time: 1.15 s


# saving as csv

In [29]:
date= '03082018'

pd.DataFrame(vaso_dose_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vaso_dose_df.csv' %(date))
pd.DataFrame(ventsettings_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date))

pd.DataFrame(echodata_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_echodata_df.csv' %(date))
pd.DataFrame(weightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_weightfirstday_df.csv' %(date))
pd.DataFrame(heightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_heightfirstday_df.csv' %(date))
pd.DataFrame(labs_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_labs_all_nosummary_df.csv' %(date))
pd.DataFrame(vitals_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_df.csv' %(date))
pd.DataFrame(gcsall_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcsall_df.csv' %(date))
pd.DataFrame(rrt_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_rrt_all_df.csv' %(date))
pd.DataFrame(urine_output_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_urine_output_all_df.csv' %(date))
pd.DataFrame(uti_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_uti_all_df.csv' %(date))
pd.DataFrame(bg_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_bg_all_nosummary_df.csv' %(date))

time: 2min 21s


time: 5.08 s


#### ventilator data qc

In [55]:
#icustay_id: 267059.0  test case
#t_0= 2182-11-12
#t_0+72 = 2182-11-15
#t_end_consec = 2183-11-15

#vaso_dose_df.loc[vaso_dose_df.loc[:,'icustay_id']==267059.0,:].sort_values('starttime')
vaso_dose_df.loc[vaso_dose_df.loc[:,'icustay_id']==267059.0,'starttime'].describe() #Timestamp('2182-11-19 20:55:00')
#Timestamp('2182-11-19 20:55:00')
#Timestamp('2183-01-15 04:00:00)

###takeaway: looks like ventilator data is in proper format where we are getting all instances of administration throughout a patient's icustay. 

count                     116
unique                    106
top       2183-01-15 04:00:00
freq                        3
first     2182-11-19 20:55:00
last      2183-01-15 04:00:00
Name: starttime, dtype: object

time: 8.8 ms


## updated here through 08/03/18

# clinical variable QC

In [103]:
lab_bg_overlap_list=list(labsFirstDayDf[labsFirstDayDf['subject_id'].isin(list(
    bgFirstDayDf['subject_id'].unique()))]['subject_id'].unique())

time: 22.1 ms


In [131]:
#bgFirstDayDf
#(labsFirstDayDf[labsFirstDayDf['subject_id'].isin(lab_bg_overlap_list)])
(labsFirstDayDf[labsFirstDayDf['subject_id']==99912])[['potassium_min','potassium_max','icustay_id','subject_id','hadm_id']]

,potassium_min,potassium_max,icustay_id,subject_id,hadm_id
61504,3.9,4.6,236183,99912,189380


time: 10.1 ms


In [130]:
#5-9-18
#(bgFirstDayDf[bgFirstDayDf['subject_id']==(lab_bg_overlap_list)])
#(bgFirstDayDf[bgFirstDayDf['subject_id']==4])[['bicarbonate','specimen','charttime']]
(bgFirstDayDf[bgFirstDayDf['subject_id']==99912])[['potassium','specimen','charttime','icustay_id','subject_id','hadm_id']]
#bgFirstDayDf
#99966 and 99912 subject id has potassium


#qc, checking overlapping categories between bgFirstDayDf and labsFirstDatDf
#subject_id has potassium 4.7 on 2191-08-23 21:38:00 in bgFirstDayDf 
#subject_id has potassium 4.4-4.9 in labsFirstDayDf. 


(ie.intime - interval '6' hour) AND (ie.intime + interval '1' day)


#labs: # -- This query pivots lab values taken in the first 24 hours of a patient's stay
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit.
#these are from labevents
#labs taken between 6 hours prior to admin and 1 day after admission. 

#bloodgas: # -- The aim of this query is to pivot entries related to blood gases and
# -- chemistry values which were found in LABEVENTS
#labs taken between 6 hours prior to admin and 1 day after admission. 

#ok so labs uses itemid of either 50822 & 50971 where as bloodgas only uses 50822. 
#50822 is postassium, whole blood (category bloodgas) while 50971 is potassium in blood (category chemistry)

,potassium,specimen,charttime,icustay_id,subject_id,hadm_id
243343,4.4,None,2153-04-13 13:37:00,236183,99912,189380
243344,NaN,None,2153-04-13 16:26:00,236183,99912,189380
243345,NaN,None,2153-04-13 16:50:00,236183,99912,189380
243346,NaN,None,2153-04-13 18:36:00,236183,99912,189380
243347,NaN,None,2153-04-13 20:53:00,236183,99912,189380
243348,NaN,None,2153-04-13 21:54:00,236183,99912,189380
243349,NaN,None,2153-04-14 00:20:00,236183,99912,189380
243350,NaN,None,2153-04-14 01:50:00,236183,99912,189380
243351,NaN,None,2153-04-14 04:32:00,236183,99912,189380
243352,NaN,None,2153-04-14 06:38:00,236183,99912,189380


time: 19.2 ms
